In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
sample_submit_df = pd.read_csv('./data/submit.csv')

In [23]:
df = train_df.drop(columns=['id'])

In [24]:
df = df.dropna()

In [25]:
df.columns

Index(['loan_amnt', 'term', 'interest_rate', 'grade', 'employment_length',
       'purpose', 'credit_score', 'application_type', 'loan_status'],
      dtype='object')

In [36]:
X = df[['term', 'interest_rate', 'grade', 'purpose', 'credit_score', 'application_type']]
y = df['loan_status']

In [40]:
X = pd.get_dummies(X)
y = pd.get_dummies(y)

In [49]:
X.head()

,interest_rate,credit_score,term_3 years,term_5 years,grade_A1,grade_A2,grade_A3,grade_A4,grade_A5,grade_B1,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,application_type_Individual,application_type_Joint App
1,16.29,700.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,21.98,670.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,8.59,710.0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,13.99,680.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,7.35,790.0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0


In [43]:
y = y.drop(columns=['FullyPaid'])

In [44]:
y.head()

,ChargedOff
1,1
2,0
3,0
4,0
5,0


In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y['ChargedOff'], random_state=0)

In [51]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

/Users/takumisetoyama/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [54]:
pred = lr.predict(X_test)

print(pred)

[0 0 0 ... 0 0 0]


In [53]:
# f1_scoreのインポート
from sklearn.metrics import f1_score

# 実測値y_test,予測値predを使ってf1_scoreを計算
f_one = f1_score(y_test, pred)

# 評価結果の表示
print(f_one)

0.036509833749864176


In [55]:
from sklearn.metrics import confusion_matrix

# 実測値y_test,予測値predを使ってconfusion_matrixを作成
cm = confusion_matrix(y_test, pred)

# 混同行列の表示
print(cm)

[[36760   126]
 [ 8741   168]]
